In [ ]:
dbutils.widgets.text("catalog", "main")
dbutils.widgets.text("database", "flights_validation_dev")

Box(children=(Label(value='catalog'), Text(value='main')))

Box(children=(Label(value='database'), Text(value='flights_dev')))

## Read csv data (batch mode)

In [ ]:
# import sys, os
# sys.path.append('./src')

import os
current_dir = os.getcwd()
## This path change is needed to make this run with Databricks Workflows
if current_dir.split('/')[-1] == 'tests':
  root_dir = os.path.dirname(current_dir)
  print("Root dir:", root_dir)
  os.chdir(root_dir)
  sys.path.append(root_dir + '/src')

from flights.transforms import flight_transforms, shared_transforms
from flights.utils import flight_utils

from flights.utils import flight_utils

catalog = dbutils.widgets.get("catalog")
database = dbutils.widgets.get("database")

path = "/databricks-datasets/airlines/part-0000*"
raw_table_name = f"{catalog}.{database}.flights_raw"


In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS IDENTIFIER(:catalog || '.' || :database);

In [ ]:
df = flight_utils.read_batch(spark, path).limit(40)
display(df)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
0,1987,12,19,6,1330,1305,1452,1445,AS,84,NA,82,100,NA,7,25,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1,1987,12,20,7,1330,1305,1502,1445,AS,84,NA,92,100,NA,17,25,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
2,1987,12,21,1,1326,1305,1514,1445,AS,84,NA,108,100,NA,29,21,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
3,1987,12,22,2,1314,1305,1447,1445,AS,84,NA,93,100,NA,2,9,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
4,1987,12,23,3,1336,1305,1455,1445,AS,84,NA,79,100,NA,10,31,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
5,1987,12,24,4,1332,1305,1454,1445,AS,84,NA,82,100,NA,9,27,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
6,1987,12,26,6,1305,1305,1443,1445,AS,84,NA,98,100,NA,-2,0,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,NO
7,1987,12,27,7,1314,1305,1506,1445,AS,84,NA,112,100,NA,21,9,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
8,1987,12,29,2,1410,1305,1543,1445,AS,84,NA,93,100,NA,58,65,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
9,1987,12,30,3,1315,1305,1443,1445,AS,84,NA,88,100,NA,-2,10,PDX,SFO,550,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,YES


## Transform data

In [5]:
df_transformed = (
        df.transform(flight_transforms.delay_type_transform)
          .transform(shared_transforms.add_metadata_columns)
    )

## Write raw Delta Lake table (batch mode)

In [ ]:
df_transformed.write.format("delta").mode("overwrite").saveAsTable(raw_table_name)
print(f"Succesfully wrote data to {raw_table_name}")

Succesfully wrote data to main.flights_dev.flights_raw
